In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
#!pip install catboost
from catboost import CatBoostClassifier
from sklearn.utils import class_weight

In [ ]:
#Reading the Data Files
train=pd.read_csv('../input/jobathon-analytics-vidhya/train.csv')
print(train.shape)
train.head()

In [ ]:
test=pd.read_csv('../input/jobathon-analytics-vidhya/test.csv')
print(test.shape)
test.head()

**EDA**

In [ ]:
sns.countplot(x='Response',data=train)
plt.show()

In [ ]:
sns.countplot(x='Accomodation_Type',data=train,hue='Response')
plt.show()

In [ ]:
sns.countplot(x='Health Indicator',data=train,hue='Response')
plt.show()

**Filling missing values**

In [ ]:
train.isnull().sum()

In [ ]:
cols_with_missing = [col for col in train.columns if train[col].isnull().any()]
print('Columns with Missing values : ')
cols_with_missing

In [ ]:
#Adding new columns to give the information about missing rows
for col in cols_with_missing:
  train[col + '_was_missing'] = train[col].isnull()
  train[col + '_was_missing']=train[col + '_was_missing'].apply(lambda x: 1 if x==True else 0)

In [ ]:
cols_with_missing = [col for col in test.columns if test[col].isnull().any()]

In [ ]:
for col in cols_with_missing:
  test[col + '_was_missing'] = test[col].isnull()
  test[col + '_was_missing']=test[col + '_was_missing'].apply(lambda x: 1 if x==True else 0)

In [ ]:
#Filling missing values with a new category 'X'

train['Health Indicator'].fillna('X',inplace=True)
test['Health Indicator'].fillna('X',inplace=True)

train['Holding_Policy_Duration'].fillna('X',inplace=True)
test['Holding_Policy_Duration'].fillna('X',inplace=True)

train['Holding_Policy_Type'].fillna('X',inplace=True)
test['Holding_Policy_Type'].fillna('X',inplace=True)

In [ ]:
train.isnull().sum().sum()

**Feature Engineering**

In [ ]:
#Making the column to be a categorical

train['Holding_Policy_Type']=train['Holding_Policy_Type'].apply(lambda x:str(x))
test['Holding_Policy_Type']=test['Holding_Policy_Type'].apply(lambda x:str(x))

#Making the column to be a categorical

train['Reco_Policy_Cat']=train['Reco_Policy_Cat'].apply(lambda x:str(x))
test['Reco_Policy_Cat']=test['Reco_Policy_Cat'].apply(lambda x:str(x))


#Getting a new feature age_diff
train['age_diff']=train['Upper_Age']-train['Lower_Age']
test['age_diff']=test['Upper_Age']-test['Lower_Age']

In [ ]:
#Making a copies of original data

train_org=train.copy()
test_org=test.copy()

In [ ]:
def response_coding(data,column,alpha):
  '''This function performs reponse coding on the given column with laplace smoothing'''
  responses={}
  train=train_org
  unique_categories=list(train[column].unique())
  for category in unique_categories:
    prob_score=0
    if 1 in list(train[train[column]==category]['Response'].value_counts().index):
      prob_score=(train[train[column]==category]['Response'].value_counts()[1]+alpha*1)/(len(train[train[column]==category]['Response'])+alpha*2)
    else:
      prob_score=(alpha*1)/(len(train[train[column]==category]['Response'])+alpha*2)
    responses[category]=prob_score
  data[column+'_response']=data[column].apply(lambda x : responses[x] if x in responses.keys() else 0 )

In [ ]:
#Getting the reponse coded columns

response_coding_columns=['Region_Code','City_Code','Health Indicator','Holding_Policy_Duration','Reco_Policy_Cat','age_diff','Reco_Policy_Premium']
for col in response_coding_columns:
  response_coding(train,col,1)
  response_coding(test,col,1)

In [ ]:
#Performing One hot Encoding on Categoricla columns
category_labels_dict={}
category_columns=['Holding_Policy_Type','Reco_Policy_Cat','City_Code','Health Indicator']
for column in category_columns:
  unq_categories=list(train[column].unique())
  #dictionary=dict((value,index) for index,value in enumerate(unq_categories) )
  category_labels_dict[column]=unq_categories

In [ ]:
def get_dummies(data,column):
  category_labels=category_labels_dict[column]
  for category in category_labels:
    data[column+'_'+category]=data[column].apply(lambda x:1 if x==category else 0)
  data.drop([column],axis=1,inplace=True)

In [ ]:
for column in category_columns:
  get_dummies(train,column)
  get_dummies(test,column)

In [ ]:
# Converting Holding_Policy_Duration into a numeric column

train['Holding_Policy_Duration']=train['Holding_Policy_Duration'].replace('14+','15')
test['Holding_Policy_Duration']=test['Holding_Policy_Duration'].replace('14+','15')
train['Holding_Policy_Duration']=train['Holding_Policy_Duration'].apply(lambda x: float(x) if x!='X' else 0 )
test['Holding_Policy_Duration']=test['Holding_Policy_Duration'].apply(lambda x: float(x) if x!='X' else 0 )

In [ ]:
#Performing Label encoding on some columns
train['Accomodation_Type']=train['Accomodation_Type'].apply(lambda x : 1 if x=='Rented' else 0)
test['Accomodation_Type']=test['Accomodation_Type'].apply(lambda x : 1 if x=='Rented' else 0)

train['Reco_Insurance_Type']=train['Reco_Insurance_Type'].apply(lambda x : 1 if x=='Joint' else 0)
test['Reco_Insurance_Type']=test['Reco_Insurance_Type'].apply(lambda x : 1 if x=='Joint' else 0)

train['Is_Spouse']=train['Is_Spouse'].apply(lambda x : 1 if x=='Yes' else 0)
test['Is_Spouse']=test['Is_Spouse'].apply(lambda x : 1 if x=='Yes' else 0)

In [ ]:
#Creating a new column  is_joint_and_married
train['is_joint_and_married']=train['Reco_Insurance_Type']^train['Is_Spouse']
test['is_joint_and_married']=test['Reco_Insurance_Type']^test['Is_Spouse']

In [ ]:
train.head()

In [ ]:
test.head()

**Training**

In [ ]:
X_train=train.drop(['ID','Response'],axis=1)
y_train=train[['Response']]
X_test=test.drop(['ID'],axis=1)

In [ ]:
#Creating train and test split
X_train, X_cv, y_train, y_cv = train_test_split(X_train, y_train,stratify=y_train, test_size=0.3, random_state=42)

In [ ]:
print('X_train : ',X_train.shape,' y_train : ',y_train.shape)
print('X_cv : ',X_cv.shape,' y_cv : ',y_cv.shape)
print('X_test : ',X_test.shape)

In [ ]:
scalar=StandardScaler()
columns_std=['City_Code_response','Region_Code_response','Upper_Age','Lower_Age','Health Indicator_response','Holding_Policy_Duration','Region_Code',
             'Holding_Policy_Duration_response','Reco_Policy_Premium','age_diff','Reco_Policy_Cat_response','age_diff_response','Reco_Policy_Premium_response']
scalar.fit(X_train[columns_std])

In [ ]:
X_train[columns_std]=scalar.transform(X_train[columns_std])
X_cv[columns_std]=scalar.transform(X_cv[columns_std])
X_test[columns_std]=scalar.transform(X_test[columns_std])

In [ ]:
weights = class_weight.compute_class_weight('balanced',np.unique(y_train['Response']),y_train['Response'])
print('Using class weights while training')
list(weights)

In [ ]:
#Training catboost Classifier model
params={
    'loss_function':'Logloss',
    'eval_metric':'AUC',
    'verbose':200,
    'random_seed':42,
    'class_weights':list(weights)
}

cat_boost=CatBoostClassifier(**params,)
cat_boost.fit(X_train,y_train,eval_set=(X_cv,y_cv),use_best_model=True,plot=True)

In [ ]:
print('Train AUC :',roc_auc_score(y_train, cat_boost.predict_proba(X_train)[:, 1]))
print('CV AUC :',roc_auc_score(y_cv, cat_boost.predict_proba(X_cv)[:, 1]))

In [ ]:
feat_imp=[t for t in zip(X_train.columns,cat_boost.get_feature_importance())]
feat_imp_df=pd.DataFrame(feat_imp,columns=['feature','importance'])
feat_imp_df=feat_imp_df.sort_values('importance',ascending=False)
print('Most important Features are :')
feat_imp_df=feat_imp_df[feat_imp_df['importance']>=1]
sns.barplot(x='importance',y='feature',data=feat_imp_df)
plt.show()

In [ ]:
def predict(model,filename):
  '''This function takes model and filename and gives predictions csv file'''
  predicted_proba=model.predict_proba(X_test)
  predicted_proba=np.asarray(predicted_proba)
  predicted_proba=predicted_proba[:,1]
  predictions=pd.DataFrame(columns=['ID','Response'])
  predictions['ID']=test['ID']
  predictions['Response']=predicted_proba
  predictions.to_csv(filename,index=False)
  return predictions

In [ ]:
predictions=predict(cat_boost,'submission.csv')

In [ ]:
predictions.head()